### CIS 4360 Stock Recommendation Based on DCF

#### Instructor: Alex Pang
#### Student: Tu Ha

In [23]:
import datetime
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import sqlite3
import yahoofinancials
import plotly.graph_objects as go

In [24]:
import option
import utils

In [25]:

from stock import Stock
from DCF_model import DiscountedCashFlowModel


**Question: Use this notebook to run your DCF model for the stock that you recommend. Plot the stock price of that stock and possibly include a few Moving Averages and/or RSI**

In [26]:
#getting the Fair value of XOM 

eps5YData = {'XOM':0.427}
    
symbol = 'XOM'

# default option
opt = option.Option()
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlitedb/Equity.db")

db_file = opt.sqlite_db
db_connection = sqlite3.connect(db_file)
#2023, 10, 1
as_of_date = datetime.date(2023, 10, 1)

stock = Stock(opt, db_connection, symbol)
stock.load_financial_data()

model = DiscountedCashFlowModel(stock, as_of_date)

print("Shares ", stock.get_num_shares_outstanding())
print("FCC ", stock.get_free_cashflow())
beta = stock.get_beta()
wacc = stock.lookup_wacc_by_beta(beta)
print("Beta ", beta)
print("WACC ", wacc)
print("Total debt ", stock.get_total_debt())
print("cash ", stock.get_cash_and_cash_equivalent())

eps5y = eps5YData[symbol]


model.set_FCC_growth_rate(eps5y, eps5y/2, 0.04)

model_price = model.calc_fair_value()
print(f"DCF price for {symbol} as of {as_of_date} is {model_price}")





Loading financial data for XOM
Shares  3963293617.201056
FCC  37834000000.0
Beta  1.044
WACC  0.065
Total debt  41193000000.0
cash  29640000000.0
DCF price for XOM as of 2023-10-01 is 1128.6524923420172


In [27]:
# Plotting the  graph
from TA import *


start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2023, 10, 1)
df = stock.get_daily_hist_price(start_date, end_date)

fig = go.Figure(data=go.Scatter(x=df.index,y=df['Close'], mode='lines'))



periods = [9, 20, 50, 100, 200]
smas = SimpleMovingAverages(df, periods)
smas.run()
s1 = smas.get_series(9)
s2 = smas.get_series(50)
s3 = smas.get_series(100)
s4 = smas.get_series(200)

#storing the smas into the df
df['s1'] = s1
df['s2'] = s2
df['s3'] = s3
df['s4'] = s4


candlestick = go.Candlestick(
                            x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name = "candlestick"
                            )


#plotting the smas
Sma1 = go.Scatter(x=df.index,
                y=df["s1"],
                yaxis="y1",
                name="9 SMA"
                )
Sma2 = go.Scatter(x=df.index,
                y=df["s2"],
                yaxis="y1",
                name="50 SMA"
                )
Sma3 = go.Scatter(x=df.index,
                y=df["s3"],
                yaxis="y1",
                name="100 SMA"
                )
Sma4 = go.Scatter(x=df.index,
                y=df["s4"],
                yaxis="y1",
                name="200 SMA"
                )





fig = go.Figure(data=[candlestick, Sma1, Sma2, Sma3, Sma4])

fig.show()


